# Testing NOBIL data

### connecting with duckdb

In [ ]:
import duckdb
from pathlib import Path


path_duckdb = Path.cwd().parent / "data_warehouse"

with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.csmd_table_dump;")
    df_csmd = cursor.df()


df_csmd


In [ ]:
for kommun in df_csmd["municipality"].unique():
    print(kommun)

In [ ]:
with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.status_online_table_dump;")
    df_st = cursor.df()
    
df_st

In [ ]:
with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.connector_table_dump;")
    df_conn = cursor.df()
    
df_conn

In [ ]:
from datetime import date
import datetime

x = datetime.datetime.now()

today_date = date(x.year, x.month, x.day).isoformat()

print(today_date)


In [ ]:
import requests 
from constants.utils import API_KEY_NOBIL


params = {
    "apikey": API_KEY_NOBIL,
    "countrycode": "SWE",
    "format": "json",
    "file": "false",
}

url = "https://nobil.no/api/server/datadump.php"

r = requests.get(url=url, params=params)
r.raise_for_status()

In [ ]:
data = r.json()

In [ ]:
data["chargerstations"][0][""]

In [ ]:
from pathlib import Path
import pandas as pd
path = Path.cwd().parents[0]/"data"



df = pd.read_excel(path/"trafikanalys.xlsx", sheet_name= "Data")





In [ ]:
df.head()

In [ ]:

df = df.reset_index().rename({"index": "id"}, axis=1)
df["id"] = df["id"].apply(lambda x: x+1)




In [ ]:
df 

In [ ]:

df = df.to_dict(orient= "records")



In [ ]:
df

# Trafikanalys filter kommun och län


In [ ]:
import duckdb
from pathlib import Path


path_duckdb = Path.cwd().parent / "data_warehouse"

with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    
    df= conn.execute("select * from staging.trafikanalys;")
    
    df_trafikanalys = df.df()
    
    

In [ ]:

# ~df_trafikanalys, "~" betyder Not, så det som inte är lika med det i contains kommer det att printas ut
# astype för sätta data typ, str vektoresering för att använda strip på varje post, str med contains är vektoresera varje post och använder
# contains med regex, case är boolesk om det ska vara case sensitive och regex boolesk för tolka det man skickar in är regular expression. 
filter = df_trafikanalys[~df_trafikanalys["kommun"].astype(str).str.strip().str.contains(r"Okänd kommun|Totalt", case= False, regex= True, na= False)]



In [ ]:

kommuner = sorted(filter["kommun"].dropna().unique().tolist())






In [ ]:
import numpy as np
np.set_printoptions(threshold=np.inf)


print(kommuner)

In [ ]:

# Samma som för kommun fast nu är det för län
df_filter_län = df_trafikanalys[~df_trafikanalys["l_n"].astype(str).str.strip().str.contains(r"Okänt län|Totalt",case= False, regex= True, na=False)]





In [ ]:
df_filter_län

In [ ]:
län = sorted(df_filter_län["l_n"].dropna().unique().tolist())

In [ ]:
print(län)

# Create csv file in seed for kommun and län

In [ ]:



df_trafikanalys = df_trafikanalys[~df_trafikanalys["kommun"].astype(str).str.strip().str.contains(r"Okänd kommun|Totalt", case= False, regex= True, na= False)]
df_trafikanalys = df_trafikanalys[~df_trafikanalys["l_n"].astype(str).str.strip().str.contains(r"Okänt län|Totalt",case= False, regex= True, na=False)]



In [ ]:

# Create csv file in seeds
path = Path().cwd().parent/"data_transform/seeds"

df_kommun_lan = (df_trafikanalys[["kommun", "l_n"]].drop_duplicates())

df_kommun_lan.to_csv(path/"municipality_lan.csv", index=False, encoding= "utf-8")


In [ ]:
len(df_kommun_lan["l_n"].unique())